# Backcasting Analysis for Transitions
Backcasting is the science of taking a desired end goal and iteratively calculating the sequence of choices which lead to it. For example if we take the end goal as a bike friendly Prague, what decisions have to be made today to reach that end goal?

### Why is this analysis so Quantitative ?
Conventional scenario analysis is often centered around narratives which break up key uncertainities. This is a time consuming process and has some important limitations. One is that it can gets very complex for more than a few uncertainities as the number of possibilties grows explosively. Two it becomes hard to keep track of all the possible decisions.

For example if we take a simplified version of this model for biking in Prague :
-  Make decisions once in 4 years.
-  6 Possible decisions or Policy Levers : Increasing Policy Coordination, Anti-Car Propaganda, Raise Prices Public Transport, Invest in Bike Infra, Lobby for Biking Finance, Levy Tax on Parking
-  Each decision can have an effect on the share of biking. But decisions also interact with each other. For example increasing policy coordination will enhance the effectiveness of all policy measures. Anti-Car Propoganda can increase the chances of un-popular measures like parking taxes.

Now if we take our end year as 2040. We have 3 sets of decisions to be made. 2^6 Possible decisions each period, for 3 periods that represents 262,144 possible transition paths. Now we have to find the ones which are successful.This is where a computational approach comes handy. We can use computers to check a vast proportion of those possible paths and precisely evaluate which ones are the best to a pre-defined criteria. In our case we have a utility function.

It also becomes possible to specify the interactions amongst decisions and delayed effects of certain decision choices. Last we can use empirical data and expert judgement to estimate the key effects and probabilties involved.


### How It Works
1. Initial Conditions: We start with the current modal split (biking, cars, public transport, and others) and set future goals for biking share.
2. Action Definition: We define a set of possible policy actions, each with specific effects, limitations, and interactions.
Mean Field Approximation: This crucial component models the aggregate response of all other agents in the system (e.g., car manufacturers, oil companies, status quo supporters) to our policy actions and changing modal shares.
State Transition Exploration: The script systematically explores all possible combinations of actions over time, creating a comprehensive map of potential futures.
Path Identification: It identifies paths that reach the desired biking share goals, considering both direct effects of actions and the Mean Field response.
6. Optimization: The script determines the most efficient path - the one that reaches the goal with the fewest actions, considering both effectiveness and societal pushback.
Visualization: It generates a graph showing the evolution of biking share over time along the optimal path, including the timing of policy actions.

### User Inputs and Their Effects
You can change various inputs to explore different scenarios:

TIME_PERIODS: Adjust the number of time steps in your analysis.

initial_modal_shares: Change the starting percentages for each transportation mode.

final_states: Modify the target biking share percentages. We have used 4% as the baseline, with 7% being realistic and 20% as the dream.

ACTIONS: You can define the full set of actions that the city can take. Note however this dramatically increases the model complexity and it might take much longer to find a solution.
For each action, you can adjust:
- effects: How much it changes each mode's share. For example raising public transport fares might increase car and bike shares.
- cooldown: How long before the action can be used again.
- one_time: Whether the action can only be used once.
- success_probability: The chance the action will work.
- interactions: How this action affects the success of other actions.
- costs: Each action can have an associated cost, for example investing in biking infrastructure is expensive and might not pay off.
- MEAN_FIELD_EFFECTS: This calculates the aggregate reaction of all other agents in the system to our policy response. In our case they represent the status quo who want to retain the current car centric modal share.

- utility_function: Change how the "goodness" of a particular modal split is calculated.Both our city and our opposition(MEAN_FEILD_EFFECTS) value the modal shift differently and have different costs for their respective actions.

- EXOGENOUS_FACTORS: Add external influences that happen at specific times. Not yet implemented but factors like E-bikes, EU policy, weather, Pandemics affect the state transtitions and can be modelled by their probabiliy distributions.


### Install all required packages to the IJulia Kernel

In [5]:
using Random
using StatsBase
using DataStructures
using Plots
using Random
using Combinatorics
using Measures

### User Input Section 

In [6]:
# Time periods
const TIME_PERIODS = [1, 2, 3, 4]

# Define actions and their indices
const DECISIONS = ["IPC", "ACP", "RPPT", "IBI", "LBF", "LTP"]
const DECISION_INDICES = Dict(name => idx for (idx, name) in enumerate(DECISIONS))

# Initial modal shares (should sum to 1)
const initial_modal_shares = Dict("s" => 0.01, "c" => 0.50, "p" => 0.40, "o" => 0.09)

# Desired final biking modal shares
const final_states = [0.15, 0.10, 0.07]

# Action definitions
struct Action
    name::String
    effects::Dict{String, Float64}
    cooldown::Int
    one_time::Bool
    success_probability::Float64
    interactions::Dict{String, Float64}
    costs::Dict{String, Float64}
end

function Action(name; effects=Dict{String, Float64}(), cooldown=0, one_time=false, success_probability=1.0, interactions=Dict{String, Float64}(), costs=Dict{String, Float64}())
    effects = _normalize_effects(effects)
    return Action(name, effects, cooldown, one_time, success_probability, interactions, costs)
end

function _normalize_effects(effects::Dict{String, Float64})
    total_effect = sum(values(effects))
    if total_effect != 0
        unspecified_modes = setdiff(Set(["s", "c", "p", "o"]), Set(keys(effects)))
        adjustment = -total_effect / length(unspecified_modes)
        for mode in unspecified_modes
            effects[mode] = adjustment
        end
    end
    return effects
end

function evaluate_success_probability(action::Action, executed_actions)
    probability = action.success_probability
    for (interaction_action, adjustment) in action.interactions
        if get(executed_actions, interaction_action, 0) == 1
            probability += adjustment
        end
    end
    return clamp(probability, 0.0, 1.0)
end

function calculate_cost(action::Action, executed_actions)
    return action.costs["base"]
end

# Define the actions with their parameters
const ACTIONS = Dict(
    "IPC" => Action("IPC", cooldown=1, costs=Dict("base" => 0.01)),
    "ACP" => Action("ACP", effects=Dict("s" => 0.01, "c" => -0.005), cooldown=1, costs=Dict("base" => 0.01)),
    "RPPT" => Action("RPPT", effects=Dict("p" => -0.005, "s" => 0.0025, "c" => 0.0025), one_time=true, success_probability=0.2, interactions=Dict("IPC" => 0.2, "LBF" => 0.2), costs=Dict("base" => 0.1)),
    "IBI" => Action("IBI", effects=Dict("s" => 0.02, "c" => -0.01), one_time=true, success_probability=0.2, interactions=Dict("IPC" => 0.2, "LBF" => 0.2), costs=Dict("base" => 0.1)),
    "LBF" => Action("LBF", cooldown=1, success_probability=0.2, interactions=Dict("ACP" => 0.2), costs=Dict("base" => 0.01)),
    "LTP" => Action("LTP", effects=Dict("c" => -0.007, "s" => 0.0035, "p" => 0.0035), one_time=true, success_probability=0.2, interactions=Dict("ACP" => 0.2, "IPC" => 0.2), costs=Dict("base" => 0.01))
)

const MEAN_FIELD_EFFECTS = Dict(
    "base" => Dict("effects" => Dict("c" => 0.000, "p" => 0.000), "probability" => 0.8),
    "reactions" => Dict(
        "ACP" => Dict("effects" => Dict("c" => 0.002), "probability" => 0.7),
        "LTP" => Dict("effects" => Dict("c" => 0.002, "p" => 0.002), "probability" => 0.6),
        "RPPT" => Dict("effects" => Dict("c" => 0.0025, "s" => -0.0025), "probability" => 0.5)
    )
)

# Utility function (with normalized bike share gain)
function utility_function(path)
    if isempty(path)
        return -Inf  # Return negative infinity for empty paths
    end
    
    target_biking_share = minimum(final_states)
    initial_biking_share = initial_modal_shares["s"]
    max_possible_gain = target_biking_share - initial_biking_share
    
    total_utility = 0.0
    total_cost = 0.0
    
    for (i, step) in enumerate(path)
        modal_shares = step["modal_shares"]
        bike_share_gain = modal_shares["s"] - initial_biking_share
        normalized_gain = bike_share_gain / max_possible_gain
        step_utility = normalized_gain
        total_utility += step_utility

        # Calculate cost for this step
        step_cost = sum(get(ACTIONS[DECISIONS[i]].costs, "base", 0.0) for (i, d) in enumerate(step["decisions"]) if d == 1; init=0.0)
        total_cost += step_cost
    end
    
    # Normalize cost to be between 0 and 1
    max_possible_cost = sum(action.costs["base"] for action in values(ACTIONS)) * length(path)
    normalized_cost = total_cost / max_possible_cost
    
    # Combine normalized utility and cost (you can adjust the weights if needed)
    return 0.7 * total_utility - 0.3 * normalized_cost
end

# Exogenous factors (if any)
const EXOGENOUS_FACTORS = Dict(
    1 => Dict("effects" => Dict("s" => 0.01, "c" => -0.005, "p" => -0.0025, "o" => -0.0025), "probability" => 0.7, "label" => "E-Bikes"),
    2 => Dict("effects" => Dict("s" => 0.01, "c" => -0.005, "p" => -0.0025, "o" => -0.0025), "probability" => 0.01, "label" => "Pandemic"),
    3 => Dict("effects" => Dict("s" => 0.05, "c" => -0.025, "p" => -0.0125, "o" => -0.0125), "probability" => 0.05, "label" => "EU Biking Directive"),
    4 => Dict("effects" => Dict("s" => 0.01, "p" => -0.002, "c" => -0.004, "o" => -0.004), "probability" => 0.8, "label" => "Shared Biking"),
    5 => Dict("effects" => Dict("s" => 0.5, "p" => 0.3, "c" => -0.4, "o" => -0.4), "probability" => 1e-4, "label" => "Tipping point Social Perception")
)

# Add this function before the state_transition function

function mean_field_response(modal_shares, decisions)
    mf_effects = Dict("s" => 0.0, "c" => 0.0, "p" => 0.0, "o" => 0.0)
    
    # Apply base effects
    if rand() < MEAN_FIELD_EFFECTS["base"]["probability"]
        for (mode, delta) in MEAN_FIELD_EFFECTS["base"]["effects"]
            mf_effects[mode] += delta
        end
    end
    
    # Apply reaction effects based on our decisions
    for (action_name, reaction) in MEAN_FIELD_EFFECTS["reactions"]
        idx = DECISION_INDICES[action_name]
        if decisions[idx] == 1
            if rand() < reaction["probability"]
                for (mode, delta) in reaction["effects"]
                    mf_effects[mode] += delta
                end
            end
        end
    end
    
    return mf_effects
end


mean_field_response (generic function with 1 method)

### Mathematical Formulation of the Backcasting Analysis Model

#### 1. Indices and Sets

- Time periods: $t \in T = \{0, 1, 2, ..., T_{max}\}$
- Actions: $a \in A = \{\text{IPC, ACP, RPPT, IBI, LBF, LTP}\}$
   - IPC : Incease Policy Coordination 
   - ACP : Anti-Car Propoganda
   - RPPT : Raise Prices Public Transport 
   - IBI : Invest in Biking Infrastructure
   - LBF : Lobby for Biking Finance 
   - LTP : Levy Taxes on Car Parking 
- Transportation modes: $m \in M = \{s, c, p, o\}$ (biking, car, public transport, other)

#### 2. State Variables

- Modal shares: $x_{m,t} \in [0,1]$, where $\sum_{m \in M} x_{m,t} = 1, \forall t$
- Action availability: $A_{a,t} \in \{0,1\}$
- Cooldown timers: $C_{a,t} \in \mathbb{N}_0$
- Disabled actions: $D_t \subseteq A$

#### 3. Decision Variables

- Action execution: $y_{a,t} \in \{0,1\}$, subject to $y_{a,t} \leq A_{a,t}$

#### 4. Parameters

- Initial modal shares: $x_{m,0}$
- Target modal shares: $S_{target} = \{s_1, s_2, s_3\}$
- Action effects: $\delta_{a,m}$
- Action cooldowns: $c_a$
- Action one-time flags: $o_a \in \{0,1\}$
- Action base success probabilities: $p_a$
- Action interaction effects: $\gamma_{a,a'}$
- Action costs: $k_a$
- Mean field base effects: $\mu_{base,m}$
- Mean field reaction effects: $\mu_{a,m}$
- Exogenous factors: $E_{t,m}$ with probabilities $p_{E_t}$

#### 5. State Transition Functions

a. Modal Share Update:

$$x_{m,t+1} = x_{m,t} + \sum_{a \in A} (\eta_{a,t} \cdot y_{a,t} \cdot \delta_{a,m}) + \text{MF}_t + E_t$$

where:
- $\eta_{a,t}$ is a Bernoulli random variable with $P(\eta_{a,t} = 1) = \min(1, \max(0, p_a + \sum_{a' \in A} y_{a',t} \cdot \gamma_{a,a'}))$
- $\text{MF}_t$ represents the mean field effects
- $E_t$ represents the exogenous factors

b. Action Availability Update:

$$A_{a,t+1} = \begin{cases}
0 & \text{if } C_{a,t+1} > 0 \text{ or } a \in D_{t+1} \\
1 & \text{otherwise}
\end{cases}$$

c. Cooldown Timer Update:

$$C_{a,t+1} = \begin{cases}
c_a & \text{if } y_{a,t} = 1 \\
\max(0, C_{a,t} - 1) & \text{otherwise}
\end{cases}$$

d. Disabled Actions Update:

$$D_{t+1} = D_t \cup \{a \in A : o_a = 1 \text{ and } y_{a,t} = 1\}$$

#### 6. Mean Field Approximation

$$\text{MF}_t = \xi_{base,t} \cdot \mu_{base,m} + \sum_{a \in A} y_{a,t} \cdot \mu_{a,m} \cdot \xi_{a,t}$$

where $\xi_{base,t}$ and $\xi_{a,t}$ are Bernoulli random variables with probabilities defined in the mean field effects.

#### 7. Utility Function

For a path $p = (x_t, y_t)_{t=0}^{T_{max}}$:

$$U(p) = 0.7 \cdot \sum_{t=1}^{T_{max}} \frac{x_{s,t} - x_{s,0}}{s_{target} - x_{s,0}} - 0.3 \cdot \frac{\sum_{t=1}^{T_{max}} \sum_{a \in A} y_{a,t} \cdot k_a}{K_{max}}$$

where $s_{target} = \min(S_{target})$ and $K_{max}$ is the maximum possible total cost.

#### 8. Optimization Problem

$$\begin{aligned}
\max_{y_{a,t}} & \quad \mathbb{E}[U(p)] \\
\text{s.t.} & \quad |x_{s,T_{max}} - s| < \epsilon \text{ for some } s \in S_{target} \\
& \quad y_{a,t} \leq A_{a,t} \quad \forall a \in A, t \in T \\
& \quad \sum_{m \in M} x_{m,t} = 1 \quad \forall t \in T \\
& \quad 0 \leq x_{m,t} \leq 1 \quad \forall m \in M, t \in T
\end{aligned}$$

#### 9. Solution Approach

1. Graph Building and Path Finding:
   For each Monte Carlo simulation:
   
   a. Initialize the root node $v_0$ with:
      - Time $t_0 = 0$
      - Modal shares $x_{m,0}$
      - Empty decision set
      - Zero cost
      - All actions available and no cooldowns

   b. Create a queue $Q$ and enqueue $v_0$

   c. While $Q$ is not empty:
      1.   Dequeue a node $v$
      2.  If $t_v = T_{max}$:
           - If $|x_{s,v} - s| < \epsilon$ for some $s \in S_{target}$, add the path to $v$ to the set of valid paths $P$
           - Continue to the next node in $Q$
      3. Generate all possible action combinations $\mathcal{C} = \{c \subseteq A : A_{a,v} = 1 \text{ for all } a \in c\}$
      4.  For each combination $c \in \mathcal{C}$:
           - Create a new decision vector $y$ where $y_a = 1$ if $a \in c$, else $y_a = 0$
           - Apply the state transition function to get new modal shares $x'$, cooldowns $C'$, and disabled actions $D'$
           - Calculate the cost of this transition
           - Create a new node $v'$ with updated state variables
           - Add an edge from $v$ to $v'$
           - Enqueue $v'$ to $Q$

   d. From the set of valid paths $P$, select the top $k$ paths based on the utility function $U(p)$

2. Monte Carlo Simulation:
   - Perform $N$ iterations of the graph building and path finding process
   - Collect all top $k$ paths from each iteration into a set $P_{all}$

3. Best Path Selection:
   $$p^* = \underset{p \in P_{all}}{\text{argmax }} U(p)$$

#### 10. Complexity Analysis

- Graph building complexity per simulation: $O(|T| \cdot 2^{|A|})$
  - At each time step, we consider all possible action combinations
  - The number of nodes at each level can grow exponentially with the number of actions
- Path selection complexity per simulation: $O(|P| \cdot |T|)$, where $|P|$ is the number of valid paths
- Overall time complexity: $O(N \cdot (|T| \cdot 2^{|A|} + |P| \cdot |T|))$

where $N$ is the number of Monte Carlo simulations.

Note: In practice, the actual runtime may be lower due to pruning of infeasible paths and early termination when target states are reached.

### Core Simulation Logic

1. **State Representation**:
   The simulation uses a `StateNode` struct to represent each state. This includes:
   - The current time period
   - Modal shares for different transportation modes (biking, car, public transport, other)
   - Decisions made to reach this state
   - Parent node reference
   - Children nodes
   - Accumulated cost
   - Debug log
   - Cooldowns for actions
   - Set of disabled actions

2. **Graph Building**:
   The `build_graph` function constructs a tree of possible states:
   - It starts with a root node representing the initial modal shares.
   - For each time step, it generates all possible combinations of available actions.
   - It applies these actions to the current state using the `state_transition` function.
   - New states are created as child nodes, considering action cooldowns and disabled actions.
   - This process continues until reaching the final time period.

3. **State Transition**:
   The `state_transition` function simulates the effects of chosen actions:
   - It calculates the changes in modal shares based on the selected actions.
   - It considers the success probability of each action.
   - It applies exogenous effects to simulate external factors.
   - It updates cooldowns for actions and disables one-time actions.

4. **Path Collection**:
   As the graph is built, paths that reach states close to the desired final modal shares are collected:
   - A path is considered valid if its final state is within a small tolerance of any target final state.
   - Each valid path is stored as a sequence of states from the initial to the final state.

5. **Path Evaluation**:
   The `evaluate_path` function assesses the quality of each collected path:
   - It uses a utility function to score the path based on its final state and possibly other factors like cost.

6. **Best Path Selection**:
   The `find_best_path` function selects the optimal path from all collected paths:
   - It compares the evaluation scores of all paths.
   - The path with the highest score is chosen as the best path.

7. **Result Visualization**:
   The `visualize_best_path` function creates a plot of the best path:
   - It shows the biking modal share over time.
   - It annotates the plot with successful and failed actions at each time step.

8. **Output Generation**:
   The simulation produces several outputs:
   - A detailed description of the best path, including decisions made at each step and resulting modal shares.
   - A visual plot of the best path.
   - CSV files containing data on the best paths and analysis results.

This simulation approach explores all possible decision sequences within the constraints of the model, allowing for the identification of optimal strategies to increase biking modal share. It accounts for the complexities of action interactions, cooldowns, and stochastic effects in the system.


In [7]:
# Modify the state_transition function
function state_transition(modal_shares_prev, decisions, current_time)
    modal_shares = copy(modal_shares_prev)
    executed_actions = Dict(action_name => decisions[DECISION_INDICES[action_name]] for action_name in DECISIONS)
    actions_to_disable = Set()
    total_cost = 0.0
    debug_log = String[]

    for (action_name, action) in ACTIONS
        if get(executed_actions, action_name, 0) == 1
            success_prob = evaluate_success_probability(action, executed_actions)
            success = rand() <= success_prob
            if success
                push!(debug_log, "Action $action_name succeeded:")
                for (mode, delta) in action.effects
                    modal_shares[mode] += delta
                    push!(debug_log, "  $mode: $(round(delta, digits=4))")
                end
                total_cost += calculate_cost(action, executed_actions)
            else
                push!(debug_log, "Action $action_name failed")
                if action.one_time
                    push!(actions_to_disable, action_name)
                end
            end
        end
    end

    mf_effects = mean_field_response(modal_shares_prev, decisions)
    push!(debug_log, "Mean Field Effects:")
    for (mode, delta) in mf_effects
        modal_shares[mode] += delta
        push!(debug_log, "  $mode: $(round(delta, digits=4))")
    end

    # Exogenous factors
    exogenous_effects = get(EXOGENOUS_FACTORS, current_time, Dict())
    if !isempty(exogenous_effects)
        push!(debug_log, "Exogenous Effects ($(exogenous_effects["label"])):")
        if rand() < exogenous_effects["probability"]
            total_exogenous_effect = sum(values(exogenous_effects["effects"]))
            adjustment = -total_exogenous_effect / (length(modal_shares) - length(exogenous_effects["effects"]))
            for mode in keys(modal_shares)
                delta = get(exogenous_effects["effects"], mode, adjustment)
                modal_shares[mode] += delta
                push!(debug_log, "  $mode: $(round(delta, digits=4))")
            end
        else
            push!(debug_log, "  No $(exogenous_effects["label"]) effects applied (probability not met)")
        end
    end

    # Ensure modal shares are within [0,1]
    for mode in keys(modal_shares)
        if modal_shares[mode] < 0 || modal_shares[mode] > 1
            push!(debug_log, "Clipping $mode from $(round(modal_shares[mode], digits=4)) to $(round(clamp(modal_shares[mode], 0, 1), digits=4))")
        end
        modal_shares[mode] = clamp(modal_shares[mode], 0, 1)
    end

    # Normalize modal shares to sum to 1
    total_share = sum(values(modal_shares))
    if !isapprox(total_share, 1.0, atol=1e-6)
        push!(debug_log, "Adjusting modal shares to sum to 1:")
        adjustment_factor = 1 / total_share
        for mode in keys(modal_shares)
            old_share = modal_shares[mode]
            modal_shares[mode] *= adjustment_factor
            push!(debug_log, "  $mode: $(round(old_share, digits=4)) -> $(round(modal_shares[mode], digits=4))")
        end
    end

    return modal_shares, actions_to_disable, total_cost, debug_log
end

function set_random_seed(seed=nothing)
    if seed !== nothing
        Random.seed!(seed)
    end
end
# State node representation
mutable struct StateNode
    time::Int
    modal_shares::Dict{String, Float64}
    decisions::Union{Nothing, Vector{Int}}
    parent::Union{Nothing, StateNode}
    children::Vector{StateNode}
    cost::Float64
    debug_log::Vector{String}
    cooldowns::Dict{String, Int}
    disabled_actions::Set{String}
end

function StateNode(time, modal_shares; decisions=nothing, parent=nothing, cooldowns=nothing, disabled_actions=nothing, cost=0.0, debug_log=String[])
    if cooldowns === nothing
        cooldowns = Dict(action => 0 for action in DECISIONS)
    else
        cooldowns = copy(cooldowns)
    end
    if disabled_actions === nothing
        disabled_actions = Set{String}()
    else
        disabled_actions = copy(disabled_actions)
    end
    return StateNode(time, copy(modal_shares), decisions, parent, StateNode[], cost, debug_log, cooldowns, disabled_actions)
end

function build_graph(initial_modal_shares, final_states)
    root = StateNode(0, initial_modal_shares)
    queue = Deque{StateNode}()
    push!(queue, root)  # Use push! to add the root node to the queue
    paths = []

    while !isempty(queue)
        current_node = popfirst!(queue)
        if current_node.time == length(TIME_PERIODS)
            if any(isapprox(current_node.modal_shares["s"], final_state, atol=0.005) for final_state in final_states)
                path = []
                node = current_node
                while node.parent !== nothing
                    push!(path, Dict(
                        "time" => node.time,
                        "decisions" => node.decisions,
                        "modal_shares" => node.modal_shares,
                        "cost" => node.cost,
                        "debug_log" => node.debug_log
                    ))
                    node = node.parent
                end
                reverse!(path)
                push!(paths, path)
            end
            continue
        end

        # Decrease cooldowns
        cooldowns_next = Dict(action => max(current_node.cooldowns[action] - 1, 0) for action in DECISIONS)

        # Determine available actions
        available_actions = [action for action in DECISIONS if cooldowns_next[action] == 0 && !(action in current_node.disabled_actions)]

        # Generate all possible combinations of available actions (power set)
        action_combinations = [collect(comb) for r in 0:length(available_actions) for comb in combinations(available_actions, r)]

        # For each combination, create the decision vector
        for action_subset in action_combinations
            decisions = zeros(Int, length(DECISIONS))
            for action in action_subset
                idx = DECISION_INDICES[action]
                decisions[idx] = 1
            end

            modal_shares_next, actions_to_disable, step_cost, debug_log = state_transition(
                current_node.modal_shares,
                decisions,
                current_node.time
            )

            # Update cooldowns and disabled actions
            cooldowns_updated = copy(cooldowns_next)
            disabled_actions_updated = copy(current_node.disabled_actions)

            # Set cooldowns and disable actions as needed
            for action in action_subset
                action_obj = ACTIONS[action]
                idx = DECISION_INDICES[action]
                # Set cooldown if action has a cooldown
                if action_obj.cooldown !== nothing
                    cooldowns_updated[action] = action_obj.cooldown
                end
                # Disable one-time actions
                if action_obj.one_time
                    push!(disabled_actions_updated, action)
                end
            end

            # Disable actions that failed
            union!(disabled_actions_updated, actions_to_disable)

            # Create child node
            child_node = StateNode(
                current_node.time + 1,
                modal_shares_next,
                decisions=decisions,
                parent=current_node,
                cooldowns=cooldowns_updated,
                disabled_actions=disabled_actions_updated,
                cost=current_node.cost + step_cost,
                debug_log=debug_log
            )
            push!(current_node.children, child_node)
            push!(queue, child_node)
        end
    end

    return paths
end

function evaluate_path(path)
    return utility_function(path)
end

function find_best_path(paths)
    if isempty(paths)
        return nothing
    end
    return argmax(evaluate_path, paths)
end

function visualize_best_path(path)
    times = [0; [step["time"] for step in path]]
    shares = [initial_modal_shares["s"]; [step["modal_shares"]["s"] for step in path]]
    
    # Configure plot with adjusted margins and larger size
    p = plot(times, shares, marker=:circle,
             xlabel="Time Period", ylabel="Biking Modal Share", 
             title="Best Path for Increasing Biking Modal Share",
             label="Biking Modal Share Path", linewidth=2, 
             size=(1000, 800), left_margin=15mm, right_margin=20mm, top_margin=15mm, bottom_margin=15mm)

    # Loop through each step to add annotations
    for (i, step) in enumerate(path)
        y_pos = step["modal_shares"]["s"]
        
        annotations = String[]
        successful_actions = String[]
        failed_actions = String[]
        exogenous_effect = nothing
        
        for log_entry in step["debug_log"]
            if contains(log_entry, "succeeded")
                action = split(log_entry)[2]
                push!(successful_actions, action)
            elseif contains(log_entry, "failed")
                action = split(log_entry)[2]
                push!(failed_actions, action)
            elseif startswith(log_entry, "Exogenous Effects")
                exogenous_effect = split(log_entry, ":")[1]
            elseif contains(log_entry, "s:") && !isnothing(exogenous_effect)
                push!(annotations, exogenous_effect)
                exogenous_effect = nothing
            end
        end
        
        if !isempty(successful_actions)
            push!(annotations, "Successful: $(join(successful_actions, ", "))")
        end
        if !isempty(failed_actions)
            push!(annotations, "Failed: $(join(failed_actions, ", "))")
        end
        
        annotation_text = join(annotations, "\n")
        
        # Add annotation only if it exists, adjust position to avoid overlap
        if !isempty(annotation_text)
            annotate!(p, step["time"], y_pos + 0.005, text(annotation_text, 6))
        end
    end

    # Adding descriptive legend items for each action type
    legend_text = ["IPC: Improve Policy Coordination",
                   "ACP: Anti-Car Propaganda",
                   "RPPT: Raise Prices Public Transport",
                   "IBI: Invest in Bike Infrastructure",
                   "LBF: Lobby for Biking Finance",
                   "LTP: Levy Tax on Car Parking"]

    # Adding dummy series for the legend
    for label in legend_text
        plot!([NaN], [NaN], label=label, linestyle=:solid, marker=:none)
    end

    # Move legend to an outer position to avoid crowding
    plot!(p, legend=:outertopright)

    # Display and save the updated plot
    display(p)
    savefig(p, "best_path_plot.png")
    println("Plot saved as 'best_path_plot.png'")
end



visualize_best_path (generic function with 1 method)

### Monte Carlo Analysis

1. **Monte Carlo Simulation Process**:
   The `monte_carlo_analysis` function is implemented in the `Backcasting_run.jl` file. It works as follows:
   - The function takes a parameter `num_simulations` (set to 300 in the main function).
   - It runs the `run_single_simulation` function multiple times (300 in this case).
   - Each run of `run_single_simulation` builds a new graph and finds the top 10 best paths.
   - All these best paths from each simulation are collected into a single list.

2. **Analysis of Results**:
   The collected best paths are analyzed for:
   - **Action frequencies**
   - **Cost distribution** (average, max, and min costs)
   - **Risk categorization** (high-risk vs. low-risk paths based on cost)
   - Results are saved to CSV files for further analysis.

3. **Best Path Selection**:
   - From all the collected paths across all simulations, the single best path is selected.
   - This path is then visualized and its details are printed.

4. **Reasons for Using Monte Carlo Analysis**:
   - **Handling Stochasticity**:
     - The simulation includes several stochastic elements, such as action success probabilities, exogenous effects on modal shares, and mean-field approximations of other players' actions.
     - Monte Carlo analysis allows us to explore how these random factors affect outcomes over many iterations.
   - **Robustness of Strategies**:
     - By running multiple simulations, we can identify strategies that perform well consistently across different random scenarios, rather than relying on a single, potentially lucky outcome.
   - **Exploration of Rare Events**:
     - Some important outcomes might be rare. Running many simulations increases the chance of capturing these less common but potentially significant scenarios.
   - **Uncertainty Quantification**:
     - Monte Carlo analysis provides a distribution of outcomes, allowing us to quantify the uncertainty in our predictions and strategies.
   - **Sensitivity Analysis**:
     - By comparing results across many simulations, we can assess how sensitive our outcomes are to small changes in initial conditions or random factors.
   - **Improved Decision Making**:
     - The aggregated results from many simulations provide a more comprehensive view of possible outcomes, leading to more informed decision-making.
   - **Risk Assessment**:
     - The analysis of cost distributions and risk categories across multiple simulations allows for a more thorough assessment of the risks associated with different strategies.
   - **Validation of Model Behavior**:
     - Running many simulations helps validate that the model behaves as expected across a wide range of scenarios.


In [8]:
function run_single_simulation()
    paths = build_graph(initial_modal_shares, final_states)
    return sort(paths, by=evaluate_path, rev=true)[1:min(10, length(paths))]
end

function monte_carlo_analysis(num_simulations::Int)
    best_paths = []
    for _ in 1:num_simulations
        simulation_best_paths = run_single_simulation()
        append!(best_paths, simulation_best_paths)
    end
    return best_paths
end

function save_best_paths(best_paths, filename::String)
    open(filename, "w") do io
        # Write header
        println(io, "Time,Decisions,Modal_Shares_s,Modal_Shares_c,Modal_Shares_p,Modal_Shares_o,Cost")
        
        for path in best_paths
            for step in path
                decision_names = join([DECISIONS[i] for (i, d) in enumerate(step["decisions"]) if d == 1], "|")
                modal_shares = step["modal_shares"]
                println(io, "$(step["time"]),$decision_names,$(modal_shares["s"]),$(modal_shares["c"]),$(modal_shares["p"]),$(modal_shares["o"]),$(round(step["cost"], digits=3))")
            end
            # Add a blank line between paths
            println(io)
        end
    end
end

function save_analysis_results(action_counts, total_costs, risk_categories, filename::String)
    open(filename, "w") do io
        println(io, "Category,Item,Value")
        
        for (action, count) in action_counts
            println(io, "Action Frequency,$action,$count")
        end
        
        println(io, "Cost Analysis,Average Cost,$(mean(total_costs))")
        println(io, "Cost Analysis,Max Cost,$(maximum(total_costs))")
        println(io, "Cost Analysis,Min Cost,$(minimum(total_costs))")
        
        for (category, count) in risk_categories
            println(io, "Risk Category,$category,$count")
        end
    end
end

function analyze_best_paths(best_paths)
    println("Total simulations: $(length(best_paths))")
    
    # Initialize data structures for analysis
    action_counts = Dict{String, Int}()
    total_costs = []
    risk_categories = Dict("High Risk" => 0, "Low Risk" => 0)
    
    for path in best_paths
        total_cost = 0.0
        for step in path
            # Count actions
            for (i, decision) in enumerate(step["decisions"])
                if decision == 1
                    action_name = DECISIONS[i]
                    action_counts[action_name] = get(action_counts, action_name, 0) + 1
                end
            end
            # Accumulate cost
            total_cost += step["cost"]
        end
        push!(total_costs, total_cost)
        
        # Categorize path risk
        if total_cost > 0.5  # Example threshold for high risk
            risk_categories["High Risk"] += 1
        else
            risk_categories["Low Risk"] += 1
        end
    end
    
    # Print analysis results
    println("Action Frequencies:")
    for (action, count) in action_counts
        println("  $action: $count")
    end
    
    println("\nCost Analysis:")
    println("  Average Cost: $(mean(total_costs))")
    println("  Max Cost: $(maximum(total_costs))")
    println("  Min Cost: $(minimum(total_costs))")
    
    println("\nRisk Categories:")
    for (category, count) in risk_categories
        println("  $category: $count")
    end

    # Save analysis results to a CSV file
    save_analysis_results(action_counts, total_costs, risk_categories, "analysis_results.csv")
end

function main()
    global paths
    set_random_seed()  # Remove the fixed seed

    num_simulations = 2
    best_paths = monte_carlo_analysis(num_simulations)
    
    # Save best paths to a CSV file
    save_best_paths(best_paths, "best_paths.csv")
    
    analyze_best_paths(best_paths)

    if isempty(best_paths)
        println("No feasible paths found.")
        return
    end

    best_path = find_best_path(best_paths)

    if best_path !== nothing
        println("Best Path:")
        for step in best_path
            decision_names = [DECISIONS[i] for (i, d) in enumerate(step["decisions"]) if d == 1]
            modal_shares_str = join([k * ": " * string(round(v, digits=3)) for (k, v) in step["modal_shares"]], ", ")
            println("Time $(step["time"]):")
            println("  Decisions: $(join(decision_names, ", "))")
            println("  Modal Shares: $modal_shares_str")
            println("  Cost: $(round(step["cost"], digits=3))")
            println("  Debug Log:")
            for log_entry in step["debug_log"]
                println("    $log_entry")
            end
            println()
        end
        visualize_best_path(best_path)
    else
        println("No feasible paths found.")
    end
end

if abspath(PROGRAM_FILE) == @__FILE__
    main()
end


### References

This is a relatively novel approach, I didn't do an extensive literature review but here are some foundation texts for the conceptual approaches used. 

1. **Puterman, M. L. (1994)**, *Markov Decision Processes: Discrete Stochastic Dynamic Programming*, Wiley.
   - This book provides a comprehensive introduction to Markov Decision Processes (MDP), covering both theoretical and practical aspects.

2. **Achdou, Y., Cardaliaguet, P., Delarue, F., Porretta, A., & Santambrogio, F. (2020)**, *An Introduction to Mean Field Game Theory*, *Mean Field Games: Cetraro, Italy 2019*, pp. 1-158.
   - This paper offers an in-depth introduction to Mean Field Game Theory, explaining how individual decision-making can be studied in the context of large populations.

3. **Metropolis, N., & Ulam, S. (1949)**, *The Monte Carlo Method*, *Journal of the American Statistical Association*, 44(247), pp. 335-341.
   - The foundational paper that introduces the Monte Carlo Method, a technique for evaluating probabilistic systems through random sampling.

4. **Howard, R. A. (1966)**, *Decision Analysis: Applied Decision Theory*, *Proceedings of the Fourth International Conference on Operational Research*.
   - A seminal work that lays the foundation for decision analysis, providing a systematic approach for making informed decisions under uncertainty.
